# Spatial Density Estimation for Tackle Success Using Deep Convolutional Gauccian-Mixture Ensembling (DCGMN)
#### Expected Yards Saved (EYS) & Percent Field Influence (PFI): Metrics for Tackler Efficiencies & Role Classification

# Introduction

Aside from a few beat-up punt returners, running backs, and most recently poorly protected quarterbacks, you’d be hard-pressed to find anyone who has been within throwing distance of a football stadium who wouldn’t argue that “the tackle” is what makes football, football. If eleven grown men all aiming to bring down one ball carrier is not a calculated science, I'm here to make it one - but previous sports analytics studies already suggest that it is. Research for injury prevention, for instance, is already a focus for the NFL and will continue to progress through studying tackle-related contact. What is less explored, though, is stratifying responsibility for tackles, and recognizing which players directly or indirectly contribute to saved yardage when a scramble, run, or run after catch begins.

Using an ensemble of ten convolutional Gaussian-mixture networks trained on independent play frames, I created a model (TackleNet) that predicts the spatial distribution of where the “end-of-play” will occur with confidence estimates. Additionally, with such predictions and confidences, I made two metrics, Expected Yards Saved (EYS) and Percent Field Influence (PFI). Below is a visual demonstration of the GMM result - read on if you're interested in how it is done.

<div style="text-align:center; font-weight:bold;">Spatial Density Estimation for Successful Tackle Using Deep Convolutional Gauccian-Mixture-Network (DCGMM)</div>

<div>
<center><img src="https://imgur.com/TbcKoiU.gif" width="1200px;"/><center>
</div>

# Main Points

* **The current state of sports analytics often fails to recognize the limitations of our data and the degree of prediction confidence we may have.** Limited data means limited and **varying confidence depending on the in-play situation**, and we should account for and report these varying intervals. Much like [play-call-decision analytics](https://arxiv.org/abs/2311.03490#:~:text=version%2C%20v3), metrics like those in the BDB should include model error estimation. **We estimate the variance in prediction by using ensemble model bootstrapping methods for our spatial tackle densities, which in turn give us prediction intervals for our metrics.**


* Tackling encompasses more than big hits: **valuable tackling  skills include any type of coercion  that reduces the ball carrier’s yardage** by the end of the play. This can be direct, like the highlight hits you'll see on replay, or indirect like having positioning that manipulates the ball carrier into a worse route or out of bounds. We should have ways of measuring how direct a defender’s influence is spatially, and how much he reduces yardage.


* **Spatial density estimation and the use of the subtraction method allow us to (1) recognize tacklers for contributing to reduced expected yardage and (2) identify spatial areas of tackler influence with respect to the entire area of the field.**

# Modeling Strategy

### i. TackleNet

Continuous density estimation can be difficult. Often we simplify our state space by imagining continuous spaces, like yardage on a football field, as discrete areas. While this can be useful, it loses us a lot of information and causes our model output to be high dimensional (think 54*120 gridded yard areas). Enter deep-density mixture models, which are extremely ideal for estimating a distribution of results from inputs. The general idea revolves around the concept that **we can model our output, where the tackle or end of play occurred, by modeling the parameters of an assumed mixture distribution with deep learning. In our case, the normal mixture distribution works perfectly. Why?**

<div style="text-align: center; margin-top: 40px; margin-bottom: 40px;">
  <blockquote>
    "The mechanics of defensive effort to tackle a ball carrier match the intuition of the normal mixture model. Each tackle attempt is either successful or not - if it is, then the successful tackle will occur around some mean with variance dependent on the yards after contact - if it's not, then the successful tackle will occur downfield at some other mean with variance dependent on different yards after contact."
  </blockquote>
</div>

We'll discuss our input features later, but it’s evident with a spatially dependent problem like tackle location estimation that we need features that account for such dependencies spatially. Mapping velocities and movement features via images turns out to be a great way of doing this, and allows us to embed all our information about a play instance into a vector feature set that we can use for estimating distribution parameters using convolutional autoencoding. **We can use this auto-encoded embedding of 'player movement maps' to feed into three fully connected networks for each parameter of the normal distribution**: one for the means of the mixture distribution, one for the covariance matrix (which we assume to be diagonal for some degree of simplicity), and one for the mixture probabilities. All of this comes together to make TackleNet, which is diagrammed below.

I should note that the models were only trained on pass plays behind the line of scrimmage, and on all run plays. Downfield pass plays were omitted, as discussed later, because my focus for metric creation was for tackles and linebackers, who are usually less involved in downfield plays. Metrics were not created for cornerbacks, as often tackles should not be rewarded after the catch because they are the results of blown pass coverage.

<div>
<center><img src="https://imgur.com/LXQEqpb.png" width="800px;"/><center>
</div>


### ii. Ensemble TackleNet

If we take a step back and think about the logistics of training this mixture model, we still need to fix a few issues.

1. We need to find a way not to overfit the data by training on many dependent play situations. This means, **ideally, we should only train a model on one frame per play so as to not rely on situational dependence driving our supposedly independent observations from these Gaussian densities.**

2. As foreshadowed previously, **we certainly need to find a way to quantify our density prediction confidence for any given observation, as 9 weeks of play data is admittedly only a snapshot of what we would need to be precisely confident for density prediction in all tackle situations.**

The solution of ensemble modeling (i.e. training ten different TackleNet models on bagged frame samples limited to one from each play), is a perfect solution. **Ensembling allows us to only use independent samples in model training, and also still use a much more vast portion of the dataset available to us. Additionally it allows us to pull spatial prediction intervals confidence intervals from the range of prediction densities**. As a general convienient note, **an ensemble of Gaussian mixture models is also a Gaussian mixture model, just with more mixtures**.


<div>
<center><img src="https://imgur.com/J1vnbfe.png" width="800px;"/><center>
</div>


<div>
<center><img src="https://imgur.com/jiHsoN9.png" width="800px;"/><center>
</div>

### iii. Feature Selection & Engineering


To account for the spatial correlation with the convolutional autoencoder described above, we first need to determine feature, or 'movement,' maps that will capture the variability of end-of-play tackle outcomes. To do so, I created images describing the velocities, accelerations, and positioning of the offensive and defensive groups as a whole in various directions and areas. The figure below describes such feature maps, which are weighted by both distance from the ballcarrier and distance from the area described, and make up the 8-channel input to TackleNet. Through the convolutional layers of the TacklNet autoencoder, we can pull valuable information relevant to the tackle location from these maps.


<table style='margin: 0 auto; border-collapse: collapse; width: 100%;'>
  <tr>
    <td style="text-align: center;"> <img src="https://imgur.com/Tf7S8MO.gif" alt="Drawing" style="width: 600px;"/> </td>
    <td style="text-align: center;"> <img src="https://imgur.com/9RLgF1W.png" alt="Drawing" style="width: 300px;"/> </td>
    <td style="text-align: center;"> <img src="https://imgur.com/KQKTBZC.png" alt="Drawing" style="width: 300px;"/> </td>
  </tr>
</table>


### iv. Training Notes

After initially training models with Gaussian log loss, I recognized a problem. With class imbalance problems with a relatively low sample size, often the best maximum likelihood solution is to guess the majority class most, or all, of the time - but this does little good for utility’s sake. A similar issue was occurring here, on a more continuous scale: **due to the low sample size and "class imbalance" with play yardage result (i.e. there are many short yardage gains and few breakaway plays), the maximum likelihood distributions favored placing modes near the ball carrier, and not accounting for downfield chances.**

I considered a couple of solutions here, including adding Bayesian priors to account for chance for larger gain plays, but **settled on instead using a custom "weighted" log loss to weight breakout plays more heavily (and also other play yardages that occurred less frequently). While this certainly introduces some bias to our distribution, in our small sample setting (at least for density estimation) it’s much more useful to overpredict the chance of anomaly type plays than to not assign them any probability at all.**

Additionally, the LogSumExp estimation trick was used for our log-likelihoods to make optimization convex, which is the norm for deep-density networks like ours.


<div>
<center><img src="https://imgur.com/GjAEnRX.png" width=800px;"/><center>
</div>


# Applications

### i. Metric Creation: EYS & PFI

The potential applications of TackleNet and similar methods to estimate spatial distributions of successful tackles in real time are massive, and there is a ton of unexplored directions to explore beyond the scope of this paper. Many of the ideas that enspired the creation of TackleNet include live broadcast ideas and non-metric related applications, but they can also help us stratify responsibility and tackle roles on plays. As a general note, **I wanted to create metrics oriented mainly towards helping review film effectively (i.e. visually showing where players have influence and whether or not that influence is well distributed on the feild)**, and was **less focused on quantifying how players are performing over a sustained number of weeks**. For this reason, I spend most of the rest of the paper denstrating play by play uses and correlations of the metrics I discuss below and less time on rating players over the entire season - though there is certainly potential for this in future work. I digress - lets talk metric.

At the start of this paper we said we wanted to accomplish two things: (1) **Recognize defender tendencies that contribute to saved yardage** and (2) **Identify the "spread of influence" of a defender on reducing yardage via direct contact or coersive positioning**. We define two metrics that do these things for defensive ends, tackles, and linebackers:

1. **Expected Yards Saved (EYS)**: Difference in the expected end of play yardage caused by a players inclusion in spatial tackle density prediction.
2. **Percent of Field Influence (PFI)**: The percentage of field area that a defender's inclusion in spatial tackle density estimation influences.

EYS is simple: it accounts for just one of the two dimensions predicted on the field, and  is useful in indicating whether or not a players influence is good, or poor (**i.e. a player can influence density predictions a lot, but if his influence is in favor of more yardage for the offense, its not productive for his team - this is why we need EYS to supplement field influence metrics**).


PFI, on the other hand, is much more intricate, as it uses the full prediction capacity of our TackleNet model, monitoring a player’s influence in not only the downfield direction but also laterally. It allows us to recognize when players position themselves in a way that maximizes their sway over where the ball carrier can feasibly go. Below is a diagram of how **these metrics are created using the subtraction method on spatial tackle density predictions with and without the defensive player’s movement features affecting estimated outcomes.**


<div>
<center><img src="https://imgur.com/CiBrtN9.png" width="1000px;"/><center>
</div>



### iii. Metric Validation

Basic analysis using multivariate logistic regression with both metrics as variables allows us  to assess the correlation of the above metrics with tackle involvement. The results are listed below and show that both metrics significantly relate to the probability of individual tackles, and we can interpret such results in a way that explains what a unit increase in both metrics means for the odds of a player tackling the ball carrier.

As both EYS and PFI are redefined for each defender every tenth of a second during the play, we **aggregate each continuous score for each defender by crediting them with their maximum EYS and PFI for the play**. Note that it is also worth watching the entire play to assess where players performed most poorly throughout the play as well!

1. **The odds of tackle involvement (i.e. achieving a tackle, assist, or causing a fumble) increase by 243% for each "Expected Yard Prevented"**
2. **The odds of tackle involvement increase by 6% for each additional "Percentage of the Field Influenced" (SOI).** Note that 1% of the field is approximately 60 square yards.

<div>
<center><img src="https://imgur.com/so1EacA.png" width="600px;"/><center>
</div>

### iv. Valuable Defender Identification for Individual Plays

In addition to being useful for reviewing particular plays that a coaching staff may be interested in, EYS & PFI are ideal for identifying plays that exemplify player tendencies that worked well - or terribly. Below is a **list of plays where particular players achieved the highest EYS of week 9 of the 22-23 season**. 

<table style='margin: 0 auto; border: 0;'>
  <tr>
    <td style="text-align: center;"> <img src="https://imgur.com/M6jETqQ.png" alt="Drawing" style="width: 1200px;"/> </td>
  <tr>
    <td style="text-align: center;"> <img src="https://imgur.com/VLjxWuF.png" alt="Drawing" style="width: 1200px;"/> </td>
  <tr>
    <td style="text-align: center;"> <img src="https://imgur.com/9jMVv2i.png" alt="Drawing" style="width: 1200px;"/> </td>
</table>

It’s worth noting here that all of the **prediction intervals estimated are significant (i.e. intervals do not include 0), but there are many prediction frames where players have non-significant metric estimates. These plays are precisely the reason why prediction intervals are important, as we would not want to conclude the effect of player positioning if significance was not present!**

Additionally, even here, we can gauge the range of the prediction interval to see what values are in a plausible range for "field influence" and "expected yards saved" given there is variability in our particular point estimate.

# Single Frame Review Example

Below is a frame example in which potential tacklers have done a seemingly good job positioning  themselves properly to ensure the ball carrier, in this case, Travis Etienne Jr., does not achieve additional yardage. **In the play, only one player was credited with tackle involvement (Blake Martinez), but all 3 surrounding players have significant spatial influence**. With this said, this is a great example of what was previously discussed in terms of error estimation, as **expected yardage influence estimates are not significant, meaning we are not certain the spatial influence of the players is downfield or if the expected yardage would change given any one of them were individually omitted from the play situation.**


<div style="text-align:center; font-weight:bold;">Tackle Situation: Expected Yards Sved (EYS) & Percent Field Influence (PFI)</div>



<table style='margin: 0 auto; border: 0;'>
  <tr>
    <td style="text-align: center;"> <img src="https://imgur.com/i05FeS9.png" alt="Drawing" style="width: 500px;"/> </td>
    <td style="text-align: center;"> <img src="https://imgur.com/G03qx3l.png" alt="Drawing" style="width: 500px;"/> </td>
  <tr>
    <td colspan=2; style="text-align: center;"> <img src="https://imgur.com/EA3XHr7.png" alt="Drawing" style="width: 800px;"/> </td>
</table>


# Conlusion and Future Directions

This paper introduced TackleNet, a model using deep convolutional Gaussian-mixture ensembling for tackle spatial density estimation. Key findings include the creation of metrics (EYS and PFI) to quantify tackler efficiencies and spatial influence. The ensemble approach showcased real-time potential for applications beyond point estimates, but also recognizing model-based prediction error. Looking forward, there is certainly potential for a "deeper dive" into uses for locational estimation for tackle with TackleNet, including more season-long oriented performance metrics, correlations with salary cap, etc.

Training strategies, including ensemble modeling and convolutional autoencoders, highlighted the complexity of continuous density estimation, but also demonstrate that such methods are plausible even in relatively small sample contexts like this one, and will become even more stable and precise when trained on multiple seasons of plays. 

Validation results demonstrated the significance of interpretations for EYS and PFI in predicting tackle involvement. Additionally, the exploration of individual plays showcased the metrics' utility in identifying valuable defender tendencies through film.


# Code

Review code [here on GitHub](https://github.com/nickb1125/Big-Data-Bowl-Tackles).

Find my [LinkedIn here](https://www.linkedin.com/in/nbachelder/).